In [5]:
from ultralytics import YOLO
import cv2
import numpy as np
import re



In [1]:
modelCataract1 = YOLO("CataractWeight1/best.pt")
modelCataract2=YOLO("CataractWeight2/best.pt")


NameError: name 'YOLO' is not defined

In [7]:
def phase_sort_key_cataract(phase):
    match = re.search(r'(\d+)', phase)
    if match:
        phase_number = int(match.group())
    else:
        phase_number = float('inf')

    if phase.startswith('Phase7a'):
        phase_number += 0.5
        
    return phase_number


In [8]:

def remove_smaller_elements(lst, last_value):
    index = 0
    while index < len(lst) and lst[index] < last_value:
        index += 1

    del lst[:index]


In [26]:
def refine_phase_boundaries_cataract(predictions,frame_count,frame_rate):
    last_value=0
    phase_groups = {}
    for phase, second in predictions:
        if phase not in phase_groups:
            phase_groups[phase] = []
        phase_groups[phase].append(second)

    refined_predictions = []


    if 'Phase2' in phase_groups:
        phase2_seconds = phase_groups['Phase2']
        phase2_seconds.sort()
        diff = np.diff(phase2_seconds)
        outliers_index = np.where(diff > 60)[0]
        if len(outliers_index) > 0:
            phase_groups['Phase7a'] = phase2_seconds[outliers_index[0] + 1:]
            phase_groups['Phase2'] = phase2_seconds[:outliers_index[0] + 1]

    for phase in sorted(phase_groups.keys(), key=phase_sort_key_cataract):
        phase_seconds = phase_groups[phase]
        if not phase_seconds:
            continue


        remove_smaller_elements(phase_seconds,last_value)
        
        print(phase_seconds)
        phase_seconds.sort()

        q1, q3 = np.percentile(phase_seconds, [25, 75])

        iqr = q3 - q1

        lower_bound = q1 - 0.2 * iqr
        upper_bound = q3 + 0.2 * iqr

        refined_seconds = [second for second in phase_seconds if lower_bound <= second <= upper_bound]

        print(q1,q3)
        print(lower_bound,upper_bound)

        last_value=refined_seconds[-1]

        if not refined_seconds:
            continue

        print(refined_seconds)

        refined_predictions.append((phase, refined_seconds[0], refined_seconds[-1]))

    for i in range(len(refined_predictions) - 1):
        current_phase_end = refined_predictions[i][2]
        next_phase_start = refined_predictions[i + 1][1]
        time_gap = (next_phase_start - current_phase_end) / 2

        if time_gap % 1.0 == 0:
            refined_predictions[i] = [refined_predictions[i][0], refined_predictions[i][1], current_phase_end + time_gap-1]
            refined_predictions[i + 1] = [refined_predictions[i + 1][0], next_phase_start - time_gap ,
                                           refined_predictions[i + 1][2]]
        else:
            refined_predictions[i] = [refined_predictions[i][0], refined_predictions[i][1],
                                       math.floor(current_phase_end + time_gap)]
            refined_predictions[i + 1] = [refined_predictions[i + 1][0], math.ceil(next_phase_start - time_gap),
                                           refined_predictions[i + 1][2]]

    refined_predictions[0][1] = 0.0
    refined_predictions[-1][2] = math.ceil(frame_count / frame_rate)

    return refined_predictions


In [10]:
def formatTextCataract(model_predictions):

    phase_tools = {
        "Phase1": ["Bonn Forceps , Primary Knife , Secondary Knife"],
        "Phase2": ["Rycroft Cannula , Visco Cannula"],
        "Phase3": ["Cap Cystotome , Cap Forceps"],
        "Phase4": ["Hydro Cannula"],
        "Phase5": ["Phaco Handpiece"],
        "Phase6": ["A/I Handpiece"],
        "Phase7": ["Rycroft Cannula , Visco Cannula"],
        "Phase7a": ["Hydro Cannula , Visco Cannula"], 
        "Phase8": ["Micromanipulator , Lens Injector"],
        "Phase9": ["Micromanipulator , A/I Handpiece"],
        "Phase10": ["Rycroft Cannula , Visco Cannula"],
    }

    phase_eye_parts = {
        "Phase1": "Cornea, Iris, Cataract Lens , Sclera",  # Replace with actual eye parts for each phase
        "Phase2": "Cornea, Iris, Cataract Lens ,Sclera ",
        "Phase3": "Cornea, Iris, Lens Fragments, Sclera",
        "Phase4": "Cornea, Iris, Lens Fragments",
        "Phase5": "Cornea, Lens Fragments, Sclera",
        "Phase6": "Cornea, Capsule, Sclera",
        "Phase7": "Cornea, Capsule, Sclera",
        "Phase7a": "Cornea, Sclera",
        "Phase8": "Cornea, Artificial Lens, Sclera",
        "Phase9": "Cornea, Artificial Lens, Sclera",
        "Phase10": "Cornea, Artificial Lens, Sclera",
    }

    tools_per_phase = {}
    for phase_code, _, _ in model_predictions:
        tools = phase_tools.get(phase_code, [])
        tools_per_phase.setdefault(phase_code, []).extend(tools)

    template = """{{ PhaseName }} Phase: ({{ StartTime }} seconds - {{ EndTime }} seconds)\n--------------------->Tools Used: {{ Tools }}\n--------------------->Eye parts detected: {{ EyeParts }}\n"""

    phase_names = {
        "Phase1": "Incision",
        "Phase2": "Viscous agent injection",
        "Phase3": "Rhexis",
        "Phase4": "Hydrodissection",
        "Phase5": "Phacoemulsificiation",
        "Phase6": "Irrigation and aspiration",
        "Phase7": "Capsule polishing",
        "Phase7a": "Viscous agent injection",
        "Phase8": "Lens implant setting-up",
        "Phase9": "Viscous agent removal",
        "Phase10": "Tonifying and antibiotics",
    }


    finalTxt=''
    for phase_data in model_predictions:
        phase_code, start_time, end_time = phase_data


        phase_name = phase_names.get(phase_code, phase_code)

        tools = ", ".join(tools_per_phase.get(phase_code, []))

        eye_parts = phase_eye_parts.get(phase_code, "Replace with logic to extract eye parts")
    
        filled_template = template.replace("{{ PhaseName }}", phase_name)
        filled_template = filled_template.replace("{{ StartTime }}", str(int(start_time)))
        filled_template = filled_template.replace("{{ EndTime }}", str(int(end_time)))
        filled_template = filled_template.replace("{{ Tools }}", tools)
        filled_template = filled_template.replace("{{ EyeParts }}", eye_parts)

        finalTxt+=filled_template
    return finalTxt

In [11]:
def process_video_cataract(video_path):

    # return [['Phase1', 0.0, 22.0],
    # ['Phase2', 23.0, 28.0],
    # ['Phase3', 29.0, 56],
    # ['Phase4', 57, 84],
    # ['Phase5', 85, 146],
    # ['Phase6', 147, 196.0],
    # ['Phase7', 197.0, 203.0],
    # ['Phase7a', 204.0, 217.0],
    # ['Phase8', 218.0, 239],
    # ['Phase9', 240, 283.0],
    # ['Phase10', 284.0, 289]]

    cap = cv2.VideoCapture(video_path)
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    interval_seconds = 1
    interval_frames = int(frame_rate * interval_seconds)

    frame_count = 0
    output1 = []

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        frame_count += 1
        
        if frame_count % interval_frames == 0:
            result = modelCataract1(frame)
            temp = result[0]
            predicted_phases = temp.names[temp.probs.top1]
            if temp.probs.top1conf>0.9:
                output1.append((predicted_phases,frame_count/frame_rate))

    cap.release()

    refined_predictions = refine_phase_boundaries_cataract(output1,frame_count,frame_rate)

    return refined_predictions

In [12]:
refined_predictions=process_video_cataract("/home/prateek/D_Drive/UNISYS/cataract-101/cataract-101/videos/case_827.mp4")


0: 640x640 Phase7 0.59, Phase6 0.15, Phase2 0.08, Phase8 0.06, Phase4 0.03, 51.1ms
Speed: 18.4ms preprocess, 51.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 Phase7 0.36, Phase2 0.29, Phase1 0.21, Phase9 0.06, Phase6 0.04, 37.0ms
Speed: 15.5ms preprocess, 37.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 Phase7 0.55, Phase1 0.22, Phase2 0.18, Phase6 0.04, Phase3 0.01, 45.9ms
Speed: 15.1ms preprocess, 45.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 Phase7 0.87, Phase2 0.10, Phase6 0.03, Phase1 0.00, Phase3 0.00, 45.5ms
Speed: 15.6ms preprocess, 45.5ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 Phase7 0.79, Phase2 0.10, Phase6 0.07, Phase1 0.03, Phase8 0.00, 48.0ms
Speed: 16.1ms preprocess, 48.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 Phase1 0.96, Phase7 0.03, Phase8 0.00, Phase6 0.00, Phase2 0.00, 36.4ms
Speed:

IndexError: index -1 is out of bounds for axis 0 with size 0

In [17]:
cap = cv2.VideoCapture("/home/prateek/D_Drive/UNISYS/cataract-101/cataract-101/videos/case_827.mp4")
frame_rate = cap.get(cv2.CAP_PROP_FPS)
interval_seconds = 1
interval_frames = int(frame_rate * interval_seconds)

frame_count = 0
output1 = []

while True:
    ret, frame = cap.read()

    if not ret:
        break

    frame_count += 1
    
    if frame_count % interval_frames == 0:
        result = modelCataract1(frame)
        temp = result[0]
        predicted_phases = temp.names[temp.probs.top1]
        if temp.probs.top1conf>0.6:
            output1.append((predicted_phases,frame_count/frame_rate))

cap.release()


0: 640x640 Phase7 0.59, Phase6 0.15, Phase2 0.08, Phase8 0.06, Phase4 0.03, 49.0ms
Speed: 18.5ms preprocess, 49.0ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)



0: 640x640 Phase7 0.36, Phase2 0.29, Phase1 0.21, Phase9 0.06, Phase6 0.04, 69.5ms
Speed: 28.8ms preprocess, 69.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 Phase7 0.55, Phase1 0.22, Phase2 0.18, Phase6 0.04, Phase3 0.01, 70.1ms
Speed: 27.1ms preprocess, 70.1ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 Phase7 0.87, Phase2 0.10, Phase6 0.03, Phase1 0.00, Phase3 0.00, 57.6ms
Speed: 19.4ms preprocess, 57.6ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 Phase7 0.79, Phase2 0.10, Phase6 0.07, Phase1 0.03, Phase8 0.00, 60.9ms
Speed: 17.6ms preprocess, 60.9ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 Phase1 0.96, Phase7 0.03, Phase8 0.00, Phase6 0.00, Phase2 0.00, 61.3ms
Speed: 19.8ms preprocess, 61.3ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 Phase1 0.68, Phase7 0.27, Phase2 0.03, Phase9 0.02, Phase6 0.00, 62.4ms
Speed: 

In [27]:
refined_predictions = refine_phase_boundaries_cataract(output1,frame_count,frame_rate)


[6.0, 7.0, 13.0, 14.0, 19.0, 25.0, 67.0, 892.0]
11.5 35.5
6.699999999999999 40.3
[7.0, 13.0, 14.0, 19.0, 25.0]
[38.0, 40.0]
38.5 39.5
38.3 39.7


IndexError: list index out of range